<a href="https://colab.research.google.com/github/jackiet0rres/showcase/blob/main/V5SiameseNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial Emotion Recognition using Contrastive Learning (Siamese Networks)

### Download Libraries and Dataset
First we are going to download libraries we need to work with.

In [ ]:
##This is where the importations occur
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.utils import to_categorical
from google.colab import drive
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda, Dropout
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications import MobileNetV2
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.models import Model
from keras.optimizers import RMSprop
import keras.backend as K

Below is how to retrieve from my Kaggle datasets, referenced: https://github.com/bnsreenu/python_for_microscopists/blob/master/Tips_tricks_35_loading_kaggle_data_to_colab.ipynb

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
#makes it so only the owner has the permission to read and write
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d jacquelinetorres/fer-2013-disappointed

Dataset URL: https://www.kaggle.com/datasets/jacquelinetorres/fer-2013-disappointed
License(s): unknown
fer-2013-disappointed.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip /content/fer-2013-disappointed.zip

Streaming output truncated to the last 5000 lines.
  inflating: FER_2013_JT copy/train/sad/Training_65242339.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65267116.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65275626.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_6529266.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65329617.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65338712.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65338797.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65387162.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65404494.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65426218.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65430136.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65437377.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_6545735.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_65463385.jpg  
  inflating: FER_2013_JT copy/train/sad/Training_654739

### Remove corrupted files
Now, let's remove any corrupted files.

In [ ]:
from PIL import Image
filename="/content/FER_2013_JT copy"
cdir="/content/FER_2013_JT copy"
def is_valid_jpg(filename):
  try:
    i=Image.open(filename)
    return i.format =='JPEG'
  except IOError:
    return False

def remove_corrupted_images_from_folder(cdir, verbose=False):
  total=0
  counter = 0
  for subdir, dirs, files in os.walk(cdir):
      for filename in files:
          filepath = subdir + os.sep + filename
          if filepath.endswith(".jpg") or filepath.endswith(".png"):
            total += 1
            if not is_valid_jpg(filepath):
              counter +=1
              os.remove(filepath)
              if (verbose):
                print(f"--Removed file ${filepath}")
  print(f"{counter} jpeg files removed out of {total} in folder {cdir}")

test_dir = "/content/FER_2013_JT copy/test"
train_dir = "/content/FER_2013_JT copy/train"

remove_corrupted_images_from_folder(train_dir)
remove_corrupted_images_from_folder(test_dir)

0 jpeg files removed out of 47898 in folder /content/FER_2013_JT copy/train
0 jpeg files removed out of 15480 in folder /content/FER_2013_JT copy/test


### Reducing the Dataset
So for starters. I am going to reduce the dataset since having over 50,000 images is costly. We are going to have a new reduced folder for each test and train folder. I am going to make that each class in train set has 420 images while each class in the test set to have 100 images.

In [ ]:
train_og_path='/content/FER_2013_JT copy/train'
test_og_path='/content/FER_2013_JT copy/test'

###function that counts how many images there are in the folder
def count_images_by_class(folder_path):
  class_wise_counts = {}
  for class_dir in os.listdir(folder_path):
    class_path = os.path.join(folder_path, class_dir)
    if os.path.isdir(class_path):
      num_images = 0
      for root, _, files in os.walk(class_path):
        for filename in files:
          if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            num_images += 1
      class_wise_counts[class_dir] = num_images
  return class_wise_counts

###applying the function
train_counts = count_images_by_class(train_og_path)
test_counts = count_images_by_class(test_og_path)

print("Train Set - Images per Class:")
for class_name, count in train_counts.items():
  print(f"{class_name}: {count}")

print("\nTest Set - Images per Class:")
for class_name, count in test_counts.items():
  print(f"{class_name}: {count}")


Train Set - Images per Class:
angry: 3995
disappointed: 6071
sad: 4830
neutral: 4965
surprise: 3171
happy: 7215
disgust: 436
interested: 13118
fear: 4097

Test Set - Images per Class:
angry: 958
disappointed: 4210
sad: 1247
neutral: 1233
surprise: 831
happy: 1774
disgust: 111
interested: 4092
fear: 1024


In [ ]:
####these would be the folder names
original_train_dir = train_dir
original_test_dir = test_dir
reduced_train_dir = os.path.join(filename, 'reduced_train')
reduced_test_dir = os.path.join(filename, 'reduced_test')

###since the folders don't exist, we must make the directories
os.makedirs(reduced_train_dir, exist_ok=True)
os.makedirs(reduced_test_dir, exist_ok=True)

target_train_images = 420
target_test_images = 100

####want to reduce the target and train directories
def reduce_images(source_dir, target_dir, target_count):
    class_dirs = [os.path.join(source_dir, class_label) for class_label in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, class_label))]

    for class_dir in class_dirs:
        class_label = os.path.basename(class_dir)
        class_filenames = os.listdir(class_dir)
        num_images = len(class_filenames)

        ####notifies if the original directories have less images
        if source_dir == original_test_dir and num_images < target_count:
            print(f"Warning: Class {class_label} has only {num_images} images in test set (less than target {target_count})")
            continue

        ####will randomly select the images from each class to put in the reduced folder
        random.shuffle(class_filenames)
        images_to_copy = min(target_count, num_images)
        selected_images = class_filenames[:images_to_copy]

        target_class_dir = os.path.join(target_dir, class_label)
        os.makedirs(target_class_dir, exist_ok=True)  #creates the directories of each class
        for filename in selected_images:
            source = os.path.join(class_dir, filename)
            target = os.path.join(target_class_dir, filename)
            shutil.copy(source, target)

        ###helps notify me how many images were removeed to make the reduced dataset
        if source_dir == original_train_dir:
            print(f"Class {class_label}: Selected and copied {images_to_copy} images to reduced train set")
        else:
            if num_images > target_count:
                images_to_remove = num_images - target_count
                print(f"Class {class_label}: Class has {num_images} images, removing {images_to_remove} images to keep {target_count} in reduced test set")
            else:
                print(f"Class {class_label}: Selected and copied {images_to_copy} images to reduced test set (limited by available images)")

reduce_images(original_train_dir, reduced_train_dir, target_train_images)
reduce_images(original_test_dir, reduced_test_dir, target_test_images)

print("Image reduction completed (random selection for train and test sets)!")


Class angry: Selected and copied 420 images to reduced train set
Class disappointed: Selected and copied 420 images to reduced train set
Class sad: Selected and copied 420 images to reduced train set
Class neutral: Selected and copied 420 images to reduced train set
Class surprise: Selected and copied 420 images to reduced train set
Class happy: Selected and copied 420 images to reduced train set
Class disgust: Selected and copied 420 images to reduced train set
Class interested: Selected and copied 420 images to reduced train set
Class fear: Selected and copied 420 images to reduced train set
Class angry: Class has 958 images, removing 858 images to keep 100 in reduced test set
Class disappointed: Class has 4210 images, removing 4110 images to keep 100 in reduced test set
Class sad: Class has 1247 images, removing 1147 images to keep 100 in reduced test set
Class neutral: Class has 1233 images, removing 1133 images to keep 100 in reduced test set
Class surprise: Class has 831 images, 

After taking away the images and creating the new folders, let's verify the amount of images in each class to be able to see if we have the exact amount of images for all classes.

In [ ]:
###applying the function
train_counts = count_images_by_class(reduced_train_dir)
test_counts = count_images_by_class(reduced_test_dir)

print("Train Set - Images per Class:")
for class_name, count in train_counts.items():
  print(f"{class_name}: {count}")

print("\nTest Set - Images per Class:")
for class_name, count in test_counts.items():
  print(f"{class_name}: {count}")


Train Set - Images per Class:
angry: 714
disappointed: 714
sad: 714
neutral: 714
surprise: 714
happy: 714
disgust: 714
interested: 714
fear: 714

Test Set - Images per Class:
angry: 200
disappointed: 200
sad: 200
neutral: 200
surprise: 200
happy: 200
disgust: 200
interested: 200
fear: 200


Now let's remove the old test and train folders to make it less confusing.

In [ ]:
shutil.rmtree(train_dir)
shutil.rmtree(test_dir)

print("Old train and test folders deleted.")

Old train and test folders deleted.


### Renaming Image Names
Another preprocess step. Let's rename the files to have img1.jpg and so on so img[i].jpg

In [ ]:
train_path = os.path.join(filename, "reduced_train")

start_number = 1


for class_folder in os.listdir(train_path):
  class_path = os.path.join(train_path, class_folder)
  ###verifies if it is a directory
  if os.path.isdir(class_path):
    for i, filename in enumerate(os.listdir(class_path)):
      #retrieves the file extension
      base, ext = os.path.splitext(filename)
      ###new filename
      new_filename = f"{class_folder}_train{start_number + i}{ext}"
      ###path of new filename
      new_filepath = os.path.join(class_path, new_filename)
      ###path of old filename
      old_filepath = os.path.join(class_path, filename)
      ###makes sure not to overwrite existing files
      if not os.path.exists(new_filepath):
        os.rename(old_filepath, new_filepath)
        print(f"Renamed {filename} to {new_filename}")
      else:
        print(f"Skipping {filename} - File already exists with name {new_filename}")

Streaming output truncated to the last 5000 lines.
Renamed Training_66039199.jpg to disappointed_train713.jpg
Renamed Training_43058403.jpg to disappointed_train714.jpg
Skipping sad_train371.jpg - File already exists with name sad_train1.jpg
Renamed sad_train92.jpg to sad_train2.jpg
Renamed sad_train140.jpg to sad_train3.jpg
Skipping sad_train124.jpg - File already exists with name sad_train4.jpg
Skipping sad_train346.jpg - File already exists with name sad_train5.jpg
Skipping Training_30695201.jpg - File already exists with name sad_train6.jpg
Skipping sad_train222.jpg - File already exists with name sad_train7.jpg
Skipping sad_train59.jpg - File already exists with name sad_train8.jpg
Skipping Training_41182563.jpg - File already exists with name sad_train9.jpg
Skipping sad_train420.jpg - File already exists with name sad_train10.jpg
Skipping sad_train307.jpg - File already exists with name sad_train11.jpg
Skipping sad_train343.jpg - File already exists with name sad_train12.jpg
Skip

In [ ]:
test_path = "/content/FER_2013_JT copy/reduced_test"

start_number = 1

###same process here but with test dataset
for class_folder in os.listdir(test_path):
  class_path = os.path.join(test_path, class_folder)
  if os.path.isdir(class_path):
    for i, filename in enumerate(os.listdir(class_path)):
      base, ext = os.path.splitext(filename)
      new_filename = f"{class_folder}_test{start_number + i}{ext}"
      new_filepath = os.path.join(class_path, new_filename)
      old_filepath = os.path.join(class_path, filename)
      if not os.path.exists(new_filepath):
        os.rename(old_filepath, new_filepath)
        print(f"Renamed {filename} to {new_filename}")
      else:
        print(f"Skipping {filename} - File already exists with name {new_filename}")

Skipping angry_test64.jpg - File already exists with name angry_test1.jpg
Skipping angry_test92.jpg - File already exists with name angry_test2.jpg
Skipping PrivateTest_86408178.jpg - File already exists with name angry_test3.jpg
Skipping PublicTest_12296139.jpg - File already exists with name angry_test4.jpg
Skipping angry_test66.jpg - File already exists with name angry_test5.jpg
Skipping PublicTest_46181567.jpg - File already exists with name angry_test6.jpg
Skipping angry_test43.jpg - File already exists with name angry_test7.jpg
Skipping PrivateTest_44437242.jpg - File already exists with name angry_test8.jpg
Skipping angry_test5.jpg - File already exists with name angry_test9.jpg
Skipping angry_test23.jpg - File already exists with name angry_test10.jpg
Skipping PublicTest_66956737.jpg - File already exists with name angry_test11.jpg
Skipping angry_test61.jpg - File already exists with name angry_test12.jpg
Skipping angry_test50.jpg - File already exists with name angry_test13.jp

### Creation of Validation Set
So after renaming all the images. Let's create a validation folder to be able to have 30% of the training set in the validation folder.

In [ ]:
###create a new directory for the validation data
filename= '/content/FER_2013_JT copy'
validation_path = os.path.join(filename, 'validation')
os.makedirs(validation_path, exist_ok=True)

###want to split the train and move to the validation so this moves per class of the train
for emotion_class in os.listdir(train_path):
    class_path = os.path.join(train_path, emotion_class)
    validation_class_path = os.path.join(validation_path, emotion_class)
    os.makedirs(validation_class_path, exist_ok=True)

    images = os.listdir(class_path)

    num_validation_images = int(0.3 * len(images))

    validation_images = random.sample(images, num_validation_images)

    ###moves validation images to validation folder
    for img_name in validation_images:
        img_path = os.path.join(class_path, img_name)
        shutil.move(img_path, os.path.join(validation_class_path, img_name))

Now let's verify how many images are in each folder.

In [ ]:
train_counts = count_images_by_class(reduced_train_dir)
test_counts = count_images_by_class(reduced_test_dir)
val_counts = count_images_by_class(validation_path)

print("Train Set - Images per Class:")
for class_name, count in train_counts.items():
  print(f"{class_name}: {count}")

print("\nTest Set - Images per Class:")
for class_name, count in test_counts.items():
  print(f"{class_name}: {count}")

print("\nValidation Set - Images per Class:")
for class_name, count in val_counts.items():
  print(f"{class_name}: {count}")

Train Set - Images per Class:
angry: 500
disappointed: 500
sad: 500
neutral: 500
surprise: 500
happy: 500
disgust: 500
interested: 500
fear: 500

Test Set - Images per Class:
angry: 200
disappointed: 200
sad: 200
neutral: 200
surprise: 200
happy: 200
disgust: 200
interested: 200
fear: 200

Validation Set - Images per Class:
angry: 307
disappointed: 312
sad: 291
neutral: 298
surprise: 295
happy: 298
disgust: 298
interested: 312
fear: 304


### Similar and Dissimilar Folders
Now let's create a similar folder and dissimilar folders. Based on this reference: https://medium.com/@hayagriva99999/exploring-siamese-networks-for-image-similarity-using-contrastive-loss-f5d5ae5a0cc6

In [ ]:
train_path = "/content/FER_2013_JT copy/reduced_train"

###where i want to store the new folders
path_1 = '/content/sample_data'
pairs_path = os.path.join(path_1, 'pairs_folder')
os.makedirs(pairs_path, exist_ok=True)

###makes positive and negative directories
positive_pairs_dir = os.path.join(pairs_path, 'positive_pairs')
negative_pairs_dir = os.path.join(pairs_path, 'negative_pairs')
os.makedirs(positive_pairs_dir, exist_ok=True)
os.makedirs(negative_pairs_dir, exist_ok=True)

###define number of pairs per image
num_positive_pairs = 2
num_negative_pairs = 2

def preprocess_image(image_path, target_size=(224, 224)):
  img = load_img(image_path, target_size=target_size)
  img_array = img_to_array(img)

  ###data augmentation part
  datagen = ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
  )
  img_array = datagen.random_transform(img_array)  ###augmented image

  ###normalizes pixel values
  img_array = img_array.astype('float32') / 255.0

  return img_array


###gets all class labels
class_labels = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]

train_positive_image_paths1 = []
train_positive_image_paths2 = []
train_negative_image_paths1 = []
train_negative_image_paths2 = []
train_positive_labels = []
train_negative_labels = []

###loops through each image
for class_label in class_labels:
    class_dir = os.path.join(test_path, class_label)
    class_images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

    ###makes the positive pairs
    for i, image in enumerate(class_images):
        image_path = os.path.join(class_dir, image)
        next_image_index = (i + 1) % len(class_images)
        positive_image = class_images[next_image_index]

        positive_source = os.path.join(class_dir, image)
        positive_target = os.path.join(positive_pairs_dir, f'{image}_{positive_image}')
        ###moves to positive directory
        shutil.copy(positive_source, positive_target)
        ###preprocess images
        preprocess_image(positive_source)
        preprocess_image(positive_target)
        ###store image paths and labels
        train_positive_image_paths1.append(positive_source)
        train_positive_image_paths2.append(positive_target)
        train_positive_labels.append(1)

    ###makes negative pairs
    for _ in range(num_negative_pairs):
        negative_class = random.choice(class_labels)
        while negative_class == class_label:  ###ensures that the pair are from diff classes
            negative_class = random.choice(class_labels)
        negative_dir = os.path.join(test_path, negative_class)
        negative_image = random.choice([f for f in os.listdir(negative_dir) if os.path.isfile(os.path.join(negative_dir, f))])
        negative_source = os.path.join(negative_dir, negative_image)
        negative_target = os.path.join(negative_pairs_dir, f'{image}_{negative_image}')
        ###moves images to negative pairs folder
        shutil.copy(negative_source, negative_target)
        ###preprocess images
        preprocess_image(negative_source)
        preprocess_image(negative_target)
        ###store image paths and labels
        train_negative_image_paths1.append(negative_source)
        train_negative_image_paths2.append(negative_target)
        train_negative_labels.append(0)

print("Completed!")

Completed!


So now let's apply the same steps for the test set to create a better understanding.

In [ ]:
test_path = "/content/FER_2013_JT copy/reduced_test"

path_1 = '/content/sample_data'
pairs_path_test = os.path.join(path_1, 'pairs_folder_test')
os.makedirs(pairs_path_test, exist_ok=True)

positive_pairs_dir_test = os.path.join(pairs_path_test, 'positive_pairs_test')
negative_pairs_dir_test = os.path.join(pairs_path_test, 'negative_pairs_test')
os.makedirs(positive_pairs_dir_test, exist_ok=True)
os.makedirs(negative_pairs_dir_test, exist_ok=True)

num_positive_pairs = 2
num_negative_pairs = 2

def preprocess_image(image_path, target_size=(224, 224)):
  img = load_img(image_path, target_size=target_size)
  img_array = img_to_array(img)
  datagen = ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
  )
  img_array = datagen.random_transform(img_array)
  img_array = img_array.astype('float32') / 255.0

  return img_array

class_labels = [d for d in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, d))]

test_positive_image_paths1 = []
test_positive_image_paths2 = []
test_negative_image_paths1 = []
test_negative_image_paths2 = []
test_positive_labels = []
test_negative_labels = []

for class_label in class_labels:
    class_dir = os.path.join(test_path, class_label)
    class_images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

    for i, image in enumerate(class_images):
        image_path = os.path.join(class_dir, image)
        next_image_index = (i + 1) % len(class_images)
        positive_image = class_images[next_image_index]

        positive_source = os.path.join(class_dir, image)
        positive_target = os.path.join(positive_pairs_dir_test, f'{image}_{positive_image}')
        shutil.copy(positive_source, positive_target)
        preprocess_image(positive_source)
        preprocess_image(positive_target)
        test_positive_image_paths1.append(positive_source)
        test_positive_image_paths2.append(positive_target)
        test_positive_labels.append(1)

    for _ in range(num_negative_pairs):
        negative_class = random.choice(class_labels)
        while negative_class == class_label:
            negative_class = random.choice(class_labels)
        negative_dir = os.path.join(test_path, negative_class)
        negative_image = random.choice([f for f in os.listdir(negative_dir) if os.path.isfile(os.path.join(negative_dir, f))])
        negative_source = os.path.join(negative_dir, negative_image)
        negative_target = os.path.join(negative_pairs_dir_test, f'{image}_{negative_image}')
        shutil.copy(negative_source, negative_target)
        preprocess_image(negative_source)
        preprocess_image(negative_target)
        test_negative_image_paths1.append(negative_source)
        test_negative_image_paths2.append(negative_target)
        test_negative_labels.append(0)

print("Completed!")

Completed!


Last let's apply the same thing to validation set

In [ ]:
validation_path='/content/FER_2013_JT copy/validation'

path_1 = '/content/sample_data'
pairs_path_val = os.path.join(path_1, 'pairs_folder_val')
os.makedirs(pairs_path_val, exist_ok=True)

positive_pairs_dir_val = os.path.join(pairs_path_val, 'positive_pairs_val')
negative_pairs_dir_val = os.path.join(pairs_path_val, 'negative_pairs_val')
os.makedirs(positive_pairs_dir_val, exist_ok=True)
os.makedirs(negative_pairs_dir_val, exist_ok=True)

num_positive_pairs = 2
num_negative_pairs = 2

def preprocess_image(image_path, target_size=(224, 224)):
  img = load_img(image_path, target_size=target_size)
  img_array = img_to_array(img)
  datagen = ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
  )
  img_array = datagen.random_transform(img_array)
  img_array = img_array.astype('float32') / 255.0

  return img_array

class_labels = [d for d in os.listdir(validation_path) if os.path.isdir(os.path.join(validation_path, d))]

val_positive_image_paths1 = []
val_positive_image_paths2 = []
val_negative_image_paths1 = []
val_negative_image_paths2 = []
val_positive_labels = []
val_negative_labels = []

for class_label in class_labels:
    class_dir = os.path.join(validation_path, class_label)
    class_images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

    for i, image in enumerate(class_images):
        image_path = os.path.join(class_dir, image)
        next_image_index = (i + 1) % len(class_images)
        positive_image = class_images[next_image_index]
        positive_source = os.path.join(class_dir, image)
        positive_target = os.path.join(positive_pairs_dir_val, f'{image}_{positive_image}')
        shutil.copy(positive_source, positive_target)
        preprocess_image(positive_source)
        preprocess_image(positive_target)
        val_positive_image_paths1.append(positive_source)
        val_positive_image_paths2.append(positive_target)
        val_positive_labels.append(1)

    for _ in range(num_negative_pairs):
        negative_class = random.choice(class_labels)
        while negative_class == class_label:
            negative_class = random.choice(class_labels)
        negative_dir = os.path.join(validation_path, negative_class)
        negative_image = random.choice([f for f in os.listdir(negative_dir) if os.path.isfile(os.path.join(negative_dir, f))])
        negative_source = os.path.join(negative_dir, negative_image)
        negative_target = os.path.join(negative_pairs_dir_val, f'{image}_{negative_image}')
        shutil.copy(negative_source, negative_target)
        preprocess_image(negative_source)
        preprocess_image(negative_target)
        val_negative_image_paths1.append(negative_source)
        val_negative_image_paths2.append(negative_target)
        val_negative_labels.append(0)

print("Completed!")

Completed!


### Generator
We have to now create generators so the model that we eventually create will know to differentiate that there are pairs and take the first image and second image of the pair

In [ ]:
def siamese_data_generator(image_paths1, image_paths2, labels, batch_size=32):
    num_samples = len(image_paths1)
    while True:
        indices = np.random.choice(num_samples, size=batch_size, replace=False)
        batch_image_paths1 = [image_paths1[i] for i in indices]
        batch_image_paths2 = [image_paths2[i] for i in indices]
        batch_labels = [labels[i] for i in indices]

        X1 = np.array([preprocess_image(img_path) for img_path in batch_image_paths1])
        X2 = np.array([preprocess_image(img_path) for img_path in batch_image_paths2])
        y = np.array(batch_labels)

        yield [X1, X2], y

###Model Creation

Let's create the model.

First, the contrastive loss function is defined

In [ ]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = K.cast(y_true, y_pred.dtype)  ###makes y_pred is the same type as y_true
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

The model is below.

In [ ]:
def create_base_network(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    return Model(inputs=base_model.input, outputs=x)

def create_siamese_network(input_shape):
    base_network = create_base_network(input_shape)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    output_a = base_network(input_a)
    output_b = base_network(input_b)

    distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([output_a, output_b])

    output = Dense(1, activation='linear')(distance)

    siamese_model = Model(inputs=[input_a, input_b], outputs=output)
    siamese_model.compile(loss=contrastive_loss, optimizer=RMSprop(), metrics=['accuracy'])
    return siamese_model

def lr_schedule(epoch):
    initial_lr = 0.0001
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

input_shape = (224, 224, 3)
siamese_model = create_siamese_network(input_shape)
siamese_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model_6 (Functional)        (None, 256)                  1831456   ['input_11[0][0]',            
                                                          0          'input_12[0][0]']            
                                                                                                  
 lambda_3 (Lambda)           (None, 256)                  0         ['model_6[0][0]',       

Create early stopping to help prevent overfitting

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

### Apply Model to Training Set and Validation Set

In [ ]:
import itertools
batch_size = 32
num_epochs = 10


train_generator_positive = siamese_data_generator(train_positive_image_paths1, train_positive_image_paths2, train_positive_labels, batch_size=batch_size // 2)
train_generator_negative = siamese_data_generator(train_negative_image_paths1, train_negative_image_paths2, train_negative_labels, batch_size=batch_size // 2)

val_generator_positive = siamese_data_generator(val_positive_image_paths1, val_positive_image_paths2, val_positive_labels, batch_size=batch_size // 2)
val_generator_negative = siamese_data_generator(val_negative_image_paths1, val_negative_image_paths2, val_negative_labels, batch_size=batch_size // 2)

combined_train_generator = itertools.chain(train_generator_positive, train_generator_negative)
steps_per_epoch = (len(train_positive_labels) + len(train_negative_labels)) // batch_size

combined_val_generator = itertools.chain(val_generator_positive, val_generator_negative)
validation_steps = (len(val_positive_labels) + len(val_negative_labels)) // batch_size

siamese_model.fit(combined_train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, validation_data=combined_val_generator, validation_steps=validation_steps, callbacks=[early_stopping,lr_scheduler])


Epoch 1/10
56/56 [==============================] - 463s 8s/step - loss: 5.1409 - accuracy: 0.3996 - val_loss: 1.9220 - val_accuracy: 0.3088 - lr: 1.0000e-04
Epoch 2/10
56/56 [==============================] - 416s 7s/step - loss: 1.6025 - accuracy: 0.3371 - val_loss: 2.0553 - val_accuracy: 0.2074 - lr: 1.0000e-04
Epoch 3/10
56/56 [==============================] - 444s 8s/step - loss: 0.9855 - accuracy: 0.3047 - val_loss: 1.5138 - val_accuracy: 0.2779 - lr: 1.0000e-04
Epoch 4/10
56/56 [==============================] - 446s 8s/step - loss: 0.5285 - accuracy: 0.2411 - val_loss: 0.9463 - val_accuracy: 0.3522 - lr: 1.0000e-04
Epoch 5/10
56/56 [==============================] - 453s 8s/step - loss: 0.2882 - accuracy: 0.1775 - val_loss: 0.5753 - val_accuracy: 0.2515 - lr: 5.0000e-05
Epoch 6/10
56/56 [==============================] - 450s 8s/step - loss: 0.1791 - accuracy: 0.1094 - val_loss: 0.5122 - val_accuracy: 0.2949 - lr: 5.0000e-05
Epoch 7/10
56/56 [==============================] - 

### Evaluate Model to Test Set

In [ ]:
test_generator_positive = siamese_data_generator(test_positive_image_paths1, test_positive_image_paths2, test_positive_labels, batch_size=batch_size // 2)
test_generator_negative = siamese_data_generator(test_negative_image_paths1, test_negative_image_paths2, test_negative_labels, batch_size=batch_size // 2)

combined_test_generator = itertools.chain(test_generator_positive, test_generator_negative)
test_steps = (len(test_positive_labels) + len(test_negative_labels)) // batch_size

evaluation = siamese_model.evaluate(combined_test_generator, steps=test_steps)

print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])


56/56 [==============================] - 69s 1s/step - loss: 0.0751 - accuracy: 0.0100
Evaluation Loss: 0.07511305063962936
Evaluation Accuracy: 0.010044642724096775
